In [1]:
%reload_ext autoreload
%autoreload 2

import time
import torch 
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import cv2 
from utils import *
import argparse
import os 
import os.path as osp
from darknet import *
import pickle as pkl
import pandas as pd
import random

In [2]:
def Write(x, results):
    c1 = tuple(x[1:3].int())
    c2 = tuple(x[3:5].int())
    img = results
    cls = int(x[-1])
    label = "{0}".format(classes[cls])
    cv2.rectangle(img, c1, c2,0, 1)
    t_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_PLAIN, 1 , 1)[0]
    c2 = c1[0] + t_size[0] + 3, c1[1] + t_size[1] + 4
    cv2.rectangle(img, c1, c2,1, -1)
    cv2.putText(img, label, (c1[0], c1[1] + t_size[1] + 4), cv2.FONT_HERSHEY_PLAIN, 1, [225,255,255], 1);
    return img

In [3]:
batch_size = 1
confidence = 0.5
nms_thesh = 0.4
start = 0
CUDA = torch.cuda.is_available()
print(CUDA)


num_classes = 80
classes = load_classes("data/coco.names")


True


In [4]:
#Set up the neural network
print("Loading network.....")
model = Darknet('yolov3.cfg')
model.load_weights('yolov3.weights')
print("Network successfully loaded")

model.info["height"] = 608
inp_dim = int(model.info["height"])
assert inp_dim % 32 == 0 
assert inp_dim > 32

#If there's a GPU availible, put the model on GPU
if CUDA:
    model.cuda()


#Set the model in evaluation mode
model.eval()


Loading network.....
{'type': 'convolutional', 'size': '1', 'stride': '1', 'pad': '1', 'filters': '255', 'activation': 'linear'}
{'type': 'convolutional', 'size': '1', 'stride': '1', 'pad': '1', 'filters': '255', 'activation': 'linear'}
{'type': 'convolutional', 'size': '1', 'stride': '1', 'pad': '1', 'filters': '255', 'activation': 'linear'}
Network successfully loaded


Darknet(
  (module_list): ModuleList(
    (0): Sequential(
      (conv0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky0): LeakyReLU(negative_slope=0.1, inplace)
    )
    (1): Sequential(
      (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky1): LeakyReLU(negative_slope=0.1, inplace)
    )
    (2): Sequential(
      (conv2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (batch_norm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky2): LeakyReLU(negative_slope=0.1, inplace)
    )
    (3): Sequential(
      (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm3): BatchNorm2d(64, 

In [ ]:
cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    if ret:
        
        img = prep_image(frame, inp_dim)
        im_dim = [frame.shape[1], frame.shape[0]]
        im_dim = torch.FloatTensor(im_dim).repeat(1,2)
 
        
        write = 0


        if CUDA:
            im_dim = im_dim.cuda()
            img = img.cuda()

        with torch.no_grad():
            prediction = model(Variable(img), CUDA)


        prediction = write_results(prediction, confidence, num_classes, nms_thesh)

        if not write:                      #If we have't initialised output
            output = prediction  
            write = 1
        else:
            output = torch.cat((output,prediction))

        if CUDA:
            torch.cuda.synchronize()



        try:
            output
        except NameError:
            continue

        im_dim = torch.index_select(im_dim, 0, output[:,0].long())
        scaling_factor = torch.min(608/im_dim,1)[0].view(-1,1)


        output[:,[1,3]] -= (inp_dim - scaling_factor*im_dim[:,0].view(-1,1))/2
        output[:,[2,4]] -= (inp_dim - scaling_factor*im_dim[:,1].view(-1,1))/2



        output[:,1:5] /= scaling_factor

        for i in range(output.shape[0]):
            output[i, [1,3]] = torch.clamp(output[i, [1,3]], 0.0, im_dim[i,0])
            output[i, [2,4]] = torch.clamp(output[i, [2,4]], 0.0, im_dim[i,1])


        list(map(lambda x: Write(x, frame), output))

        #det_names = pd.Series(imlist).apply(lambda x: "detections/det_{}".format(x.split("/")[-1]))

        #list(map(cv2.imwrite, det_names, loaded_ims))

        torch.cuda.empty_cache()
        

        cv2.imshow('Ingram-Micro',frame)
        #cv2.imwrite('frame', output)

        #cv2.imshow('frame',frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()